1) PRODUTOS COM VENDA MENOR QUE O PREÇO DE TABELA

In [ ]:
SELECT O.ORDER_ID      ID_VENDA,
       P.PRODUCT_ID    ID_PROD,
       P.PRODUCT_NAME  PRODUTO,
       O.QUANTITY      QUANTIDADE,
       O.UNIT_PRICE    PRECO_VENDA,
       P.UNIT_PRICE    PRECO_TABELA
FROM ORDER_DETAILS O,  PRODUCTS P 
WHERE O.PRODUCT_ID = P.PRODUCT_ID 
  AND PRECO_VENDA  < PRECO_TABELA
ORDER BY PRECO_VENDA DESC;

-- AGRUPADO POR PRODUTO
SELECT COUNT(O.ORDER_ID)           TOTAL_VENDAS,
        P.PRODUCT_ID               ID_PROD,
        P.PRODUCT_NAME             PRODUTO,
       SUM(O.QUANTITY)             QTD_TOTAL,
       ROUND(SUM(O.UNIT_PRICE),2)  PRECO_VENDA,
       ROUND(SUM(P.UNIT_PRICE),2)  PRECO_TABELA,
       ROUND(SUM(P.UNIT_PRICE - O.UNIT_PRICE),2) DIFERENCA
FROM ORDER_DETAILS O, PRODUCTS P 
WHERE O.PRODUCT_ID = P.PRODUCT_ID 
AND  (O.UNIT_PRICE < P.UNIT_PRICE)
GROUP BY P.PRODUCT_ID,
         P.PRODUCT_NAME
ORDER BY PRECO_VENDA DESC;

-------------------------------------------------------------

2) VENDEDORES QUE MAIS VENDERAM EM 2022

In [ ]:
SELECT  E.FIRST_NAME || ' ' || E.LAST_NAME                  as NOME, 
    ROUND(SUM(OD.UNIT_PRICE * OD.QUANTITY - OD.DISCOUNT),2) as TOTAL 
FROM ORDER_DETAILS OD
INNER JOIN ORDERS    O ON O.ORDER_ID  = OD.ORDER_ID
INNER JOIN EMPLOYEES E ON E.EMPLOYEE_ID = O.EMPLOYEE_ID
WHERE DATE_PART('YEAR', O.ORDER_DATE) = 2022
GROUP BY NOME
ORDER BY TOTAL DESC;

-------------------------------------------------------------

3) OS 10 PRODUTOS MAIS CAROS

In [ ]:
SELECT PRODUCT_NAME PRODUTO,
       UNIT_PRICE PRECO
FROM PRODUCTS 
ORDER BY PRECO DESC 
LIMIT 10

-------------------------------------------------------------

4) DIFERENCA DAS VENDAS DE PRODUTOS ENTRE O ANO ATUAL E O ANTERIOR MOSTRANDO POR FORNECEDOR

In [ ]:
SELECT A2020.SUPPLIER_ID,
       A2020.COMPANY_NAME,
	   A2020.TOTAL A2020,
	   A2021.TOTAL A2021, 
      (A2021.TOTAL - A2020.TOTAL) TOTAL
FROM 
	(SELECT S.SUPPLIER_ID, 
	        S.COMPANY_NAME, 
	        SUM(ROUND((OD.UNIT_PRICE * OD.QUANTITY),2)) TOTAL
    FROM ORDER_DETAILS OD
	INNER JOIN ORDERS    O ON (O.ORDER_ID    = OD.ORDER_ID)
	INNER JOIN PRODUCTS  P ON (P.PRODUCT_ID  = OD.PRODUCT_ID )
	INNER JOIN SUPPLIERS S ON (S.SUPPLIER_ID = P.SUPPLIER_ID )
	WHERE DATE_PART('YEAR', O.ORDER_DATE) = 2020
	GROUP BY S.SUPPLIER_ID,S.COMPANY_NAME
	ORDER BY S.SUPPLIER_ID) A2020,

	(SELECT S.SUPPLIER_ID, 
	        S.COMPANY_NAME, 
	        SUM(ROUND((OD.UNIT_PRICE * OD.QUANTITY),2)) TOTAL
    FROM ORDER_DETAILS OD
	INNER JOIN ORDERS    O ON (O.ORDER_ID    = OD.ORDER_ID)
	INNER JOIN PRODUCTS  P ON (P.PRODUCT_ID  = OD.PRODUCT_ID )
	INNER JOIN SUPPLIERS S ON (S.SUPPLIER_ID = P.SUPPLIER_ID )
	WHERE DATE_PART('YEAR', O.ORDER_DATE) = 2021
	GROUP BY S.SUPPLIER_ID,S.COMPANY_NAME
	ORDER BY S.SUPPLIER_ID) A2021

WHERE (A2020.SUPPLIER_ID = A2021.SUPPLIER_ID)
GROUP BY A2020.SUPPLIER_ID,
         A2021.SUPPLIER_ID,
         A2020.COMPANY_NAME,
		 A2021.COMPANY_NAME,
		 A2020.TOTAL,
		 A2021.TOTAL

-------------------------------------------------------------

5) RANKING DAS CINCO CATEGRORIAS COM MAIS VENDAS AGRUPADAS POR ANO

In [ ]:
WITH RESULT AS 
(SELECT C.CATEGORY_NAME                             CATEG,             
	   DATE_PART('YEAR', O.ORDER_DATE)              ANO,
	   SUM(ROUND((OD.UNIT_PRICE * OD.QUANTITY),2))  TOTAL,
	   ROW_NUMBER() OVER (PARTITION BY ANO ORDER BY ANO, TOTAL DESC) AS RES 
FROM CATEGORIES C 
INNER JOIN PRODUCTS      P  ON (P.CATEGORY_ID = C.CATEGORY_ID)
INNER JOIN ORDER_DETAILS OD ON (OD.PRODUCT_ID = P.PRODUCT_ID)
INNER JOIN ORDERS        O  ON (O.ORDER_ID = OD.ORDER_ID)
GROUP BY CATEG, ANO
ORDER BY ANO, TOTAL DESC
LIMIT 100
),
FILTRO AS (
  SELECT CATEG, ANO, TOTAL FROM RESULT WHERE RES <=5
)
  SELECT CATEG, ANO, TOTAL FROM FILTRO